# 1. Using beautiful shop to extract data from wikipedia

In [119]:
# install bs4 
!conda install -c conda-forge bs4 --yes
#!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    future-0.18.2              |           py36_0         713 KB  conda-forge
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |    

In [145]:
import requests
from bs4 import BeautifulSoup
# Set headers to pretend a legitimate user to call API
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
headers

{'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [146]:
# url and parse url into readable html doc
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())

In [147]:
# inspect the wiki page and see that <tbody> contains the desired table
table = soup.tbody
#table

In [148]:
# convert reduced html readable doc (table) into a list
import numpy as np
import pandas as pd
toronto_data = []
list_table = table.find_all('tr')
for i in range(len(list_table)):
    text = list_table[i].get_text().strip().split('\n')
    toronto_data.append(text)
toronto_data = pd.DataFrame(toronto_data[1:], columns = toronto_data[0])
toronto_data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [149]:
print(toronto_data['Borough'].value_counts())
print('size of data before process:', toronto_data.shape)

Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64
size of data before process: (287, 3)


In [150]:
# remove the 'Not assigned' in 'Borough' field of the toronto data
toronto_data = toronto_data[toronto_data['Borough'] != 'Not assigned']

# check 'Not assigned' if in 'Neighbourhood' field 
if np.any(toronto_data['Neighbourhood'] != 'Not assigned'):
    print("okie, not having 'Not assigned' in Neighbourhood field")
print('size of data after process:', toronto_data.shape)
toronto_data.head(10)

okie, not having 'Not assigned' in Neighbourhood field
size of data after process: (210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


# 2. Generate (Lat,Long) from Postcode using geocode python 

In [151]:
# read longitude and latitude from file
df_latlong = pd.read_csv('Geospatial_Coordinates.csv') 
df_latlong.rename(columns = {'Postal Code':'Postcode'}, inplace = True)
df_latlong.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [152]:
# assign lat and long to the toronto_data
print(toronto_data.head(10))
toronto_data_final = toronto_data.merge(df_latlong, on = 'Postcode', how = 'left')
toronto_data_final.head(10)

   Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M6A        North York  Lawrence Heights
6       M6A        North York    Lawrence Manor
7       M7A  Downtown Toronto      Queen's Park
9       M9A         Etobicoke  Islington Avenue
10      M1B       Scarborough             Rouge
11      M1B       Scarborough           Malvern
13      M3B        North York   Don Mills North


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188
